# Computer Vision API example

### Instructions - please read through these :)
* This is a jupyter notebook so that means interactive coding in a browser
* You want to run each grey cell (the python, in this case, code) in order and one at a time
* Make sure each code cell has finished running (a number will appear inside the In[] when the cell is done)
* You can also see if the notebook is running by looking for a filled in circle in the upper right corner of this notebook
* Feel free to change code (even if you don't know python)
* There are many helper methods in this notebook so don't worry if you don't understand
* The cells that you should work with will be **<font color=green>indicated with a green font</font>**
* Have fun with this and save the notebook often :)

## Let's take care of some installs

In [1]:
!conda install --no-deps -c menpo opencv=2.4.11 --yes

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /home/nbcommon/anaconda2_410:
#
opencv                    2.4.11                 nppy27_0    menpo


In [2]:
! pip uninstall matplotlib --yes

Uninstalling matplotlib-2.0.0:
  Successfully uninstalled matplotlib-2.0.0
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
! pip install matplotlib -q

## Let's import some modules

In [4]:
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import json
from urlparse import urlparse
from urllib import urlencode
from urllib2 import Request, urlopen
from httplib import HTTPSConnection 

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

## Now to set some constants

* The Cognitive Services keys (grabbing the computer vision one)

In [5]:
f = urlopen('https://gist.githubusercontent.com/antriv/a6962d2c7580a0f7db4b7aabd6d768c5/raw/38a66f77c7fd0641324c8cbbff77828207041edc/config.json')
CONFIG=json.loads(f.read())
# grab key from json in config
subscription_key = CONFIG['subscription_key_computer_vision']

* **<font color=green>The Cognitive Services Computer Vision web service request url</font>**

In [6]:
# Variables

_url = 'https://api.projectoxford.ai/vision/v1.0/describe' # change the ending to 'analyze' or 'describe'
_key = subscription_key

_maxNumRetries = 10

## Helper functions

In [7]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json() ) )

        break
        
    return result

In [8]:
def renderResultOnImage( result, img ):
    """Display the obtained results onto the input image"""

    if 'description' in result:
        descrip = result['description']['captions'][0]['text']
        print(descrip)
        cv2.putText( img, descrip, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3 )
    if 'categories' in result:
        # Put a rectangle around the image to reflect accent colors
        R = int(result['color']['accentColor'][:2],16)
        G = int(result['color']['accentColor'][2:4],16)
        B = int(result['color']['accentColor'][4:],16)
        if img is not None:
            cv2.rectangle( img,(0,0), (img.shape[1], img.shape[0]), color = (R,G,B), thickness = 25 )

        categoryName = sorted(result['categories'], key=lambda x: x['score'])[0]['name']
        print(categoryName)
        cv2.putText( img, categoryName, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3 )

## Analysis of an image retrieved via URL

* **<font color=green>Place an image URL here in the `urlImage` variable (must be a good quality, large image)</font>**

Input requirements:
* Supported image formats: JPEG, PNG, GIF, BMP.
* Image file size must be less than 4MB.
* Image dimensions should be greater than 50 x 50.

In [11]:
urlImage = 'http://gb.fotolibra.com/images/previews/392412-pair-of-swans-at-stanborough-park.jpeg'
# urlImage = 'you own url'

In [12]:
from IPython.display import display, Image


# Computer Vision parameters
params = { 'visualFeatures' : 'Color,Categories'} 

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None

result = processRequest( json, data, headers, params )

if result is not None:
    # Load the original image, fetched from the URL
    arr = np.asarray( bytearray( requests.get( urlImage ).content ), dtype=np.uint8 )  
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)

    renderResultOnImage( result, img )
    
    if img is not None:
        plt.subplots(figsize=(15, 20))
        plt.imshow( img )
    else:
        img = Image(url=urlImage, embed=False)
        display(img)


a swan in the water


### EXERCISE

In [ ]:
# try changing the word 'describe' to 'analyze' in the _url (web service url for REST api) cell towards the beginning